#### Consider the NLS eqution in the semiclassical regime
\begin{align}
    i \epsilon u^{\epsilon}_t + \frac{\epsilon^2}{2}u^{\epsilon}_{xx} + |u^{\epsilon}|^2\ u^{\epsilon} & = 0, \text{on} \ [-8,8] \times (0,T] \;, \\
    %u^{\epsilon}(x,0) & = A_{0}(x)e^{\frac{i S_{0}(x)}{\epsilon}} \;,\\
    u^{\epsilon}(-8,t) & =u^{\epsilon}(8,t) \;,
\end{align}
with two initial conditons: 
 \begin{align} 
    u^{\epsilon}(x,0) & = e^{-x^2}\;,
 \end{align}
and 
 \begin{align} 
    u^{\epsilon}(x,0) & = e^{-x^2} e^{i\frac{1}{\epsilon(e^x+e^{-x})}}.
 \end{align}
 #### This code computes and saves the data for the solution convergence and cost measurement plots for both initial conditions with two values of epsilon at a final time. This data is used to create figures 4 and 5 in the manuscript.

In [12]:
# For the convergence plot and error vs. runtime plots we just use the top 2 by 2 cases for both the inital data.
"""
(a) Zero phase initla data 
Eps = [0.2,0.1], Ns = [L*32*2,L*32*4] where L = xL-xR = 16 and T = 0.8.

(a) Nonero phase initla data 
Eps = [0.2,0.1], Ns = [L*32*2,L*32*4] where L = xL-xR = 16 and T = 0.5.
""";
# So for each inital data there will be four figures. This code generates the data for those plots.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Diff_Time_Integrators import Compute_Sol_ImEx, Compute_Sol_ImEx_Relaxation, sol_nl_part, \
sol_l_part, Op_Split_Exact_Solve, ImEx_schemes, Op_Sp_Coeff, adaptive_step_local_err, Compute_Sol_ImEx_Relaxation_Adp

#### Choose the case here

In [3]:
case = 'strong_cubic_foc_nonzero_phase'; 

In [4]:
def nonlin_f(z,ep,case):
    if case == 'strong_cubic_foc_zero_phase':
        y = -z 
    elif case == 'strong_cubic_foc_nonzero_phase':
        y = -z 
    return y

def A0_fun(x,case):
    if case == 'strong_cubic_foc_zero_phase':
        y = np.exp(-x**2)
    elif case == 'strong_cubic_foc_nonzero_phase':
        y = np.exp(-x**2)
    return y

def S0_fun(x,ep,case):
    if case == 'strong_cubic_foc_zero_phase':
        y = x*0
    elif case == 'strong_cubic_foc_nonzero_phase':
        y = 1/(np.exp(x)+np.exp(-x))
    return y

def initial_cond(x,ep,case):
    ucomp = A0_fun(x,case)*np.exp(1j*S0_fun(x,ep,case)/ep)
    return np.concatenate((ucomp.real,ucomp.imag)).astype('float64')

# Stiff part of the right hand side function of the ODE system in spatial doamin
def rhsNlsStiff(u,N,xi,ep):
    ucomp = u[:N]+ 1j *u[N:]
    uhat = np.fft.fft(ucomp)
    duhat = -1j*(ep/2)*xi**2*uhat
    rhs_comp = np.fft.ifft(duhat)
    dudt = np.concatenate((rhs_comp.real,rhs_comp.imag)).astype('float64')
    return dudt

# Non stiff part of right hand side function of the ODE system in spatial doamin
def rhsNlsNonStiff(u,N,ep,case):
    ucomp = u[:N]+ (1j) *u[N:]
    rhs_comp= -(1j/ep)*nonlin_f(np.abs(ucomp)**2,ep,case)*ucomp
    dudt = np.concatenate((rhs_comp.real,rhs_comp.imag)).astype('float64')
    return dudt

def position_density(u,N):  # n(x,t)
    V = u[:N]; W = u[N:]
    return V**2+W**2

def current_density(u,N):  # J(x,t)
    V = u[:N]; W = u[N:]
    return np.multiply(V[:-1],W[1:])-np.multiply(V[1:],W[:-1])

# For multi-relaxation
def rgam(gamma,u,N,dx,ep,case,inc,inv,E_old):
    uprop = u + np.dot(gamma,inc)  
    if inv == 1:
        E_prop = np.array([eta1(uprop,N,dx)])
    elif inv == 2:
        E_prop = np.array([eta1(uprop,N,dx),eta2(uprop,N,dx,ep,case)])
    return E_prop-E_old

def eta1(u,N,dx):  # Invariant: first quantity
    V = u[:N]; W = u[N:]
    return dx*np.sum(V**2+W**2)

# def eta2(u,N,dx,ep,case):   # Invariant: second quantity
#     V = u[:N]; W = u[N:]
#     z = V[:-1]**2+W[:-1]**2
#     return np.sum((ep**2/2)*(np.diff(V)**2+np.diff(W)**2)/dx + dx*nonlin_f(z,ep,case)*z)

def eta2(u,N,dx,ep,case):  # Current density
    V = u[:N]; W = u[N:]
    return dx*np.sum(np.multiply(V[:-1],W[1:])-np.multiply(V[1:],W[:-1]))

In [5]:
dts = np.empty((2,10))
dts[0,:] = 10**np.linspace(-2,-3,10); 
dts[1,:] = 10**np.linspace(-2.5,-4,10);  # this one take long time to compute

In [6]:
if case == 'strong_cubic_foc_zero_phase':
    xL = -8; xR =8; L = xR-xL; t0 = 0;  inv = 1; 
    Eps = [0.2,0.1]; Ns = np.array([L*32*2,L*32*4]); T = 0.8; 
elif case == 'strong_cubic_foc_nonzero_phase':
    xL = -8; xR =8; L = xR-xL; t0 = 0;  inv = 1;
    Eps = [0.2,0.1]; Ns = np.array([L*32*2,L*32*4]); T = 0.5; 

In [7]:
# ImEx time-stepping methods
method_names = ['ARK32','ARK43']; Stage = [4,6]; Order = [3,4]; em_Or = [2,3]; Sch_No = [3,4]
# Operator splitting methods
OS_method_names = ['Op_Sp1','Op_Sp2','Op_Sp4']; OS_Stage = [1,2,5]; OS_Order = [1,2,4]; OS_Sch_No = [0,1,2]

In [8]:
%%time
Sol_b_ARK43 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
Sol_r_ARK43 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
Sol_OpSp2 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
Sol_OpSp4 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);

RT_b_ARK43 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
RT_r_ARK43 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
RT_OpSp2 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);
RT_OpSp4 = np.empty((len(Eps),len(dts[0,:]),), dtype=object);

ImEx_idx1 = 1; OS_idx1 = 1; OS_idx2 = 2; em_or = em_Or[ImEx_idx1]; tol = 1e-8;
rkim, rkex, c, b, bhat = ImEx_schemes(Stage[ImEx_idx1],Order[ImEx_idx1],em_Or[ImEx_idx1],Sch_No[ImEx_idx1])
OpSp2_a, OpSp2_b = Op_Sp_Coeff(OS_Stage[OS_idx1],OS_Order[OS_idx1],OS_Sch_No[OS_idx1])
OpSp4_a, OpSp4_b = Op_Sp_Coeff(OS_Stage[OS_idx2],OS_Order[OS_idx2],OS_Sch_No[OS_idx2])

for i in range(len(Eps)):
    ep = Eps[i]; N = Ns[i];
    for j in range(len(dts[0,:])):
        dt = dts[i,j]; f_stiff = rhsNlsStiff; f_non_stiff = rhsNlsNonStiff; IC = initial_cond;
        print(ep); print(dt); 
        
        # ARK baseline
        b_req_t,b_tt,b_uu = Compute_Sol_ImEx(f_stiff,f_non_stiff,IC,ep,case,xL,xR,N,t0,T,method_names[ImEx_idx1],rkim,rkex,b,c,dt)
        
        # ARK relaxation
        r_req_t,r_tt,r_uu,G,IF_1,IF_5,IF_else = Compute_Sol_ImEx_Relaxation(f_stiff,f_non_stiff,IC,ep,case,xL,xR,N,t0,T, \
                                    method_names[ImEx_idx1],rkim,rkex,b,bhat,c,inv,eta1,eta2,rgam,dt)
        
         # Strang splitting
        sp2_req_t, sp2_tt, sp2_uu = Op_Split_Exact_Solve(nonlin_f,sol_nl_part,sol_l_part,IC,ep,case,xL,xR,N,t0,T, \
                                              OS_method_names[OS_idx1],OpSp2_a,OpSp2_b,dt)
        
        # 4th order splitting
        sp4_req_t, sp4_tt, sp4_uu = Op_Split_Exact_Solve(nonlin_f,sol_nl_part,sol_l_part,IC,ep,case,xL,xR,N,t0,T, \
                                              OS_method_names[OS_idx2],OpSp4_a,OpSp4_b,dt)
 
        # storing data
        RT_b_ARK43[i,j] = b_req_t; Sol_b_ARK43[i,j] = b_uu[-1]; 
        RT_r_ARK43[i,j] = r_req_t; Sol_r_ARK43[i,j] = r_uu[-1];
        RT_OpSp2[i,j] = sp2_req_t; Sol_OpSp2[i,j] = sp2_uu[-1];
        RT_OpSp4[i,j] = sp4_req_t; Sol_OpSp4[i,j] = sp4_uu[-1];

0.2
0.01
0.2
0.007742636826811269
0.2
0.005994842503189409
0.2
0.004641588833612777
0.2
0.003593813663804626
0.2
0.0027825594022071257
0.2
0.0021544346900318843
0.2
0.0016681005372000592
0.2
0.001291549665014884
0.2
0.001
0.1
0.0031622776601683794
0.1
0.0021544346900318843
0.1
0.0014677992676220691
0.1
0.001
0.1
0.0006812920690579615
0.1
0.0004641588833612782
0.1
0.00031622776601683794
0.1
0.00021544346900318845
0.1
0.00014677992676220705
0.1
0.0001
CPU times: user 7min 11s, sys: 11min 45s, total: 18min 57s
Wall time: 20min 33s


In [9]:
import os
path = './Data/'
if not os.path.exists(path):
   os.makedirs(path)

## Saving Data

In [10]:
# saving data and information         
np.save("./Data/%s_NumSol_B_ARK43.npy"%(case), Sol_b_ARK43)
np.save("./Data/%s_NumSol_R_ARK43.npy"%(case), Sol_r_ARK43)
np.save("./Data/%s_NumSol_OpSp2.npy"%(case), Sol_OpSp2)
np.save("./Data/%s_NumSol_OpSp4.npy"%(case), Sol_OpSp4)

# saving data and information         
np.save("./Data/%s_RunTime_B_ARK43.npy"%(case), RT_b_ARK43)
np.save("./Data/%s_RunTime_R_ARK43.npy"%(case), RT_r_ARK43)
np.save("./Data/%s_RunTime_OpSp2.npy"%(case), RT_OpSp2)
np.save("./Data/%s_RunTime_OpSp4.npy"%(case), RT_OpSp4)